In [ ]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import torch
import torch.nn.functional as F
from pathlib import Path
import json
from tqdm import tqdm
import matplotlib.pyplot as plt
import imageio

# Change working directory to project root
os.chdir('../../')

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

from hydra import initialize_config_module, compose
with initialize_config_module(version_base="1.3", config_module=f"hmr4d.configs"):
    # overrides = ['exp=mas/rpd/default', f'ckpt_path=outputs/rich_motion2d/default/checkpoints/e029-s016800.ckpt', "ckpt_type=pl"]
    # overrides = ['exp=mas/rpd/amass', f'ckpt_path=outputs/amass_motion2d/default/checkpoints/e019-s161680.ckpt', "ckpt_type=pl"]
    # overrides = ['exp=mas/rpd/hml', f'ckpt_path=inputs/checkpoints/mas/hml3d_mdm_transl_s589560.ckpt', "ckpt_type=pl"]
    overrides = ['exp=mas/rpd/hml', f'ckpt_path=inputs/checkpoints/mas/hml3d_mdm_e0541-s221136.ckpt', "ckpt_type=pl", "model.network.args.stats_name=HML3D_ORTHO"]

    overrides += ["model.network.args_clip.clip_pretrained_path=inputs/checkpoints/huggingface/clip-vit-base-patch32"]    
    cfg = compose(config_name='train', overrides=overrides)

In [ ]:
from hydra.utils import instantiate
from hmr4d.utils.net_utils import load_pretrained_model

model = instantiate(cfg.model, _recursive_=False)
load_pretrained_model(model, cfg.ckpt_path, cfg.ckpt_type)
model = model.eval().cuda()

In [ ]:
import hmr4d.utils.matrix as matrix
from hmr4d.utils.camera_utils import get_camera_mat_zface
distance = torch.ones((1,))
angle = torch.rand((1,)) * 2 * torch.pi
cam_mat = get_camera_mat_zface(matrix.identity_mat()[None], distance, angle)  # 1, 4, 4
T_w2c = torch.inverse(cam_mat)[0]  # 4, 4

batch = {
    "length": 120,  # Value = F
    "bbx_lurb": torch.tensor([0, 0, 1, 1], dtype=torch.float32),  # (F, 4)
    # "gt_bi01_motion2d": bi01_motion2d.float(),  # (F, 22, 2)
    # "gt_motion": joints_pos.float(),  # (F, 22, 3)
    "text": "running to left",
    # "text": "",
    "Ts_w2c": T_w2c.float()[None],  # (1, 4, 4)
    "is_pinhole": False,  # Value = False or True
}
from torch.utils.data import default_collate
batch = default_collate([batch] * 1)


In [ ]:
batch = {k: v.cuda() if isinstance(v, torch.Tensor) else v for k, v in batch.items() }

model.generate_target = "forward_sample"
outputs = model.validation_step(batch, 0)
print(outputs.keys())

In [ ]:
from hmr4d.utils.wis3d_utils import make_wis3d, get_gradient_colors, get_const_colors, add_joints22_motion_as_lines
wis3d = make_wis3d(name="hml_model", time_postfix=True)

In [ ]:
pred_bi01_motion2d = outputs['pred_bi01_motion2d'].detach()  # (B, L, J, 2)
pred_pseudo_motion3d = F.pad(pred_bi01_motion2d, (0, 1), value=0)
for b in range(8):
    add_joints22_motion_as_lines(pred_pseudo_motion3d[b], wis3d, name=f'sample{b:02d}')

---

In [ ]:
datamodule = instantiate(cfg.data)
dataloader = datamodule.train_dataloader()
dataloader = iter(dataloader)

In [ ]:
batch = dataloader.next()
print(batch.keys())
outputs = model.validation_step(batch, 0)
print(outputs.keys())

In [ ]:
from hmr4d.utils.wis3d_utils import make_wis3d, get_gradient_colors, get_const_colors, add_joints22_motion_as_lines
wis3d = make_wis3d(name="uncond2d", time_postfix=True)

In [ ]:
pred_bi01_motion2d = outputs['pred_bi01_motion2d'].detach()  # (B, L, J, 2)
pred_pseudo_motion3d = F.pad(pred_bi01_motion2d, (0, 1), value=0)
for b in range(4):
    add_joints22_motion_as_lines(pred_pseudo_motion3d[b], wis3d, name=f'sample{b:02d}')

In [ ]:
torch.save(outputs, "tmp_outputs_4bs.pt")